<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 8 [Assessment]:** RAG Evaluation</font>

<br>

Welcome to the last notebook of the course! In the previous notebook, you integrated a vector store solution into a RAG pipeline! In this notebook, you will take that same pipeline and evaluate it using numerical RAG evaluation techniques incorporating LLM-as-a-Judge metrics!

<br>

### **Learning Objectives:**

- Learn how to integrate the techniques from prior notebooks to numerically approximate the goodness of your RAG pipeline.

- **Final Exercice**: ***By working through this notebook in the Course Environment,* you will be able to submit the coding component of the course!**

<br>

### **Questions To Think About:**

- As you go along, remember what our metrics actually represent. Should our pipeline pass these objectives? Is our judge LLM sufficient for evaluating the pipeline? Does a particular metric even matter for our use case?
- If we left the vectorstore-as-a-memory component in our chain, do you think it would still pass the evaluation? Additionally, is the evaluation useful for assessing vectorstore-as-a-memory performance? 

<br>

### **Notebook Source:**

- This notebook is part of a larger [**NVIDIA Deep Learning Institute**](https://www.nvidia.com/en-us/training/) course titled [**Building RAG Agents with LLMs**](https://www.nvidia.com/en-sg/training/instructor-led-workshops/building-rag-agents-with-llms/). If sharing this material, please give credit and link back to the original course.

<br>

### **Environment Setup:**

In [4]:
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu ragas

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")

----

<br>

## **Part 1:** Pre-Release Evaluation

In our previous notebook, we successfully combined several concepts to create a document chatbot with the aim of responsive and informative interactions. However, the diversity of user interactions necessitates comprehensive testing to truly understand the chatbot's performance. Thorough testing in varied scenarios is crucial to ensure that the system is not only robust and versatile but also aligns with user and provider expectations.

After defining your chatbot's roles and implementing the necessary features, evaluating it becomes a multi-stage process:

- **Typical Use Inspection:** Start by testing scenarios most relevant to your use case. See if your chatbot can reliably navigate discussions with limited human intervention.

    - Additionally, identify limitations or compartments that should be redirected to a human for inspection/supervision (i.e., human swap-in to confirm transactions or perform sensitive navigation) and implement those options.

- **Edge Case Inspection:** Explore the boundaries of typical use, identifying how the chatbot handles less common but plausible scenarios.

    - Before any public release, assess critical boundary conditions that could pose liability risks, such as the potential generation of inappropriate content.

    - Implement well-tested guardrails on all outputs (and possibly inputs) to limit undesired interactions and redirect users into predictable conversation flows.

- **Progressive Rollout:** Rolling out your model to a limited audience (first internal, then [A/B](https://en.wikipedia.org/wiki/A/B_testing)) and implement analytics features like usage analytics dashboards and feedback avenues (flag/like/dislike/etc).

Of these three steps, the first two can be done by a small team or an individual and should be iterated on as part of the development process. Unfortunately, this needs to be done frequently and can be prone to human error. **Luckily for us, LLMs can be used to help out with LLM-as-a-Judge formulations!**

*(Yeah, this probably isn't surprising by now. LLMs being strong is why this course is here...).*

----

<br>

## **Part 2:** LLM-as-a-Judge Formulation

In the realm of conversational AI, using LLMs as evaluators or 'judges' has emerged as a useful approach for configurable automatic testing of natural language task performance:

- An LLM can simulate a range of interaction scenarios and generate synthetic data, allowing an evaluation developer to generate targeted inputs to eliciting a range of behaviors from your chatbot.

- The chatbot's correspondence/retrieval on the synthetic data can be evaluated or parsed by an LLM and a consistent output format such as "Pass"/"Fail", similarity, or extraction can be enforced.

- Many such results can be aggregated and a metric can be derived which explains something like "% of passing evaluations", "average number of relevant details from the sources", "average cosine similarity", etc.

This idea of using LLMs to test out and quantify chatbot quality, known as [**"LLM-as-a-Judge,"**](https://arxiv.org/abs/2306.05685) allows for easy test specifications that align closely with human judgment and can be fine-tuned and replicated at scale.

**There are several popular frameworks for off-the-shelf judge formulations including:**
- [**RAGAs (short for RAG Assessment)**](https://docs.ragas.io/en/stable/), which offers a suite of great starting points for your own evaluation efforts.
- [**LangChain Evaluators**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/), which are similar first-party options with many implicitly-constructible agents.

Instead of using the chains as-is, we will instead expand on the ideas and evaluate our system with a more custom solution.

----

<br>

## **Part 3: [Assessment Prep]** Pairwise Evaluator

The following exercise will flesh out a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/comparison/pairwise_string/). 

**To prepare for our RAG chain evaluation, we will need to:**

- Pull in our document index (the one we saved in the previous notebook).
- Recreate our RAG pipeline of choice.

**We will specifically be implementing a judge formulation with the following steps:**

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

**The chain should be a simple but powerful process that tests for the following objective:**

> ***Does my RAG chain outperform a narrow chatbot with limited document access.***





**This will be the system used for the final evaluation!** To see how this system is integrated into the autograder, please check out the implementation in [`frontend/server_app.py`](frontend/server_app.py).

<br>

### **Task 1:** Pull In Your Document Retrieval Index

For this exercise, you will pull in the `docstore_index` file you created as part of your earlier notebook. The following cell should be able to load in the store as-is.

In [3]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

# The code confirms that the vector store has been successfully loaded and displays:
# The total number of document chunks.
# A sample document chunk with its title, summary, and content.

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


Constructed aggregate docstore with 369 chunks

Sample Chunk:

Paper: Mistral 7B

Summary: We introduce Mistral 7B v0.1, a 7-billion-parameter language model engineered
for superior performance and efficiency. Mistral 7B outperforms Llama 2 13B
across all evaluated benchmarks, and Llama 1 34B in reasoning, mathematics, and
code generation. Our model leverages grouped-query attention (GQA) for faster
inference, coupled with sliding window attention (SWA) to effectively handle
sequences of arbitrary length with a reduced inference cost. We also provide a
model fine-tuned to follow instructions, Mistral 7B -- Instruct, that surpasses
the Llama 2 13B -- Chat model both on human and automated benchmarks. Our
models are released under the Apache 2.0 license.

Page Body: . We measure performance on a wide variety of tasks categorized as follow:\n\u2022 Commonsense Reasoning (0-shot): Hellaswag [28], Winogrande [21], PIQA [4], SIQA [22],\nOpenbookQA [19], ARC-Easy, ARC-Challenge [9], CommonsenseQA [24]\n\u2022 World Knowledge (5-shot): NaturalQuestions [1

<br>

### **Task 2: [Exercise]** Pull In Your RAG Chain

Now that we have our index, we can recreate the RAG agent from the previous notebook! 

**Key Modifications:**
- To keep things simple, feel free to disregard the vectorstore-as-a-memory component. Incorporating it will require some more overhead and will make the exercise a bit more complicated.

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
llm = instruct_llm | StrOutputParser()

#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#####################################################################
## TODO: Pull in your desired RAG Chain. Memory not necessary

## Chain 1 Specs: "Hello World" -> retrieval_chain 
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
context_getter = (
    lambda x: docs2str(
        long_reorder.invoke(
            docstore.similarity_search(x['input'], k=3)
        )
    )
)
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

## Chain 2 Specs: retrieval_chain -> generator_chain 
##   -> {"output" : <str>, ...} -> output_puller
generator_chain = (
    chat_prompt
    | llm
)
generator_chain = {'output' : generator_chain} | RunnableLambda(output_puller)  ## GIVEN

## END TODO
#####################################################################

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting about RAG!"):
    print(token, end="")

RAG, or Retrieval-Augmented Generation, is a framework that enhances the trustworthiness and factuality of Language Models (LLMs) by retrieving relevant information from an external knowledge database and integrating it with the user's query. This process grounds LLMs on the retrieved knowledge for conditional generations. 

One interesting aspect of RAG is its two-step workflow: retrieval and generation. During the retrieval stage, RAG queries a knowledge database to find the most relevant information. In the generation stage, RAG uses the retrieved information to generate responses. 

There are also different approaches to how RAG predicts target tokens. The RAG-Sequence model uses the same retrieved document to predict each token in the sequence, while the RAG-Token model can predict each token based on a different document. This flexibility allows RAG to adapt to various knowledge-intensive NLP tasks.

(Sources: TrustRAG: Enhancing Robustness and Trustworthiness in RAG; Retrieval-A

<br>

### **Step 3:** Generating Synthetic Question-Answer Pairs

In this section, we can implement the first few part of our evaluation routine:

- **Sample the RAG agent document pool to find two document chunks.**
- **Use those two document chunks to generate a synthetic "baseline" question-answer pair.**
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [6]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', 'INPUT: {input}')])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
        " DO NOT SAY: \"Here is an interesting question pair\" or similar. FOLLOW FORMAT!"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How does TrustRAG, a robust framework, enhance the performance of Retrieval-Augmented Generation (RAG) 
systems and improve their resistance to attacks?

Answer: TrustRAG is a two-stage defense mechanism designed to strengthen the robustness and trustworthiness of RAG 
systems. It employs K-means clustering to identify potential attack patterns in retrieved documents based on their 
semantic embeddings, effectively isolating suspicious content. Additionally, it uses cosine similarity and ROUGE 
metrics to detect malicious documents and resolve discrepancies between the model's internal knowledge and external
information through a self-assessment process. TrustRAG functions as a plug-and-play, training-free module that 
integrates with any language model, maintaining high contextual relevance while providing substantial improvements 
in retrieval accuracy, efficiency, and attack resistance compared to existing approaches. It has been shown to 
improve runtime performance, achieving a runtime approximately two times greater than Vanilla RAG, which is a 
reasonable trade-off for the significant improvements in robustness and reliability offered by TrustRAG.

QA Pair 2

Question: How can gender stereotypes in movie recommender systems lead to privacy issues and what kind of broader 
implications can this have on digital computational science?

Answer: The document discusses a study that shows how gender stereotypes in movie recommender systems can lead to 
privacy issues by enabling attackers to determine users' gender, a private attribute, by exploiting these 
stereotypes and analyzing users' feedback data. The study also found that inference algorithms performed more 
effectively than relying solely on feedback data for gender inference. Additionally, the study quantified the 
extent of gender stereotypes to evaluate their broader impact on digital computational science. The document also 
mentions that demographic biases such as gender bias can be introduced into the system for business purposes, and 
this can result in issues such as filter bubbles or feedback loop effects, which can confine users to repetitive, 
similar recommendations and limit their exposure to diverse content, possibly reinforcing stereotypical behavior. 
The study concludes that movie recommender systems often contain gender stereotypes in their data, which raises 
significant privacy concerns.

QA Pair 3

Question: How does TrustRAG, a defense framework for Retrieval-Augmented Generation (RAG) systems, enhance 
robustness and trustworthiness in the face of corpus poisoning attacks?

Answer: TrustRAG is a two-stage defense mechanism aimed at countering attacks involving multiple maliciously 
injected documents in RAG systems. It employs K-means filtering to reduce the presence of malicious documents and 
uses both internal and external knowledge sources to resolve conflicts and mitigate the impact of these attacks. By
comprehensively evaluating TrustRAG across benchmarks, it has been demonstrated that it outperforms existing 
defenses, maintaining high accuracy even under aggressive poisoning scenarios where traditional approaches fail. 
TrustRAG's plug-and-play architecture allows for immediate deployment with any Large Language Models (LLMs).

<br>

### **Step 4:** Answer The Synthetic Questions

In this section, we can implement the third part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use the RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [7]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    rag_answer = rag_chain.invoke(q)
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

QA Pair 1
Question: How does TrustRAG, a robust framework, enhance the performance of Retrieval-Augmented Generation (RAG) 
systems and improve their resistance to attacks?

RAG Answer: TrustRAG is a robust framework designed to enhance the performance of Retrieval-Augmented Generation 
(RAG) systems and improve their resistance to attacks. According to the abstract of "TrustRAG: Enhancing Robustness
and Trustworthiness in RAG," the framework aims to address the vulnerabilities of RAG systems to corpus poisoning 
attacks, which can significantly degrade the performance of large language models (LLMs) through the injection of 
malicious content.

TrustRAG implements a two-stage defense mechanism to filter compromised and irrelevant content before it reaches 
the language model. The first stage employs K-means clustering to identify potential attack patterns in retrieved 
documents based on their semantic embeddings, effectively isolating suspicious content.

The second stage involves the use of an objective function to evaluate the quality of the answer produced by the 
LLM based on the k retrieved texts for the target question. The objective function is large when the answer 
produced by the LLM is the target answer, indicating that the retrieved texts are relevant and trustworthy.

According to the paper, TrustRAG conducts an extensive evaluation on multiple knowledge databases and LLMs, 
comparing it with advanced RAG systems. The results show that TrustRAG decreases the attack success rate while 
maintaining high response accuracy on different popular RAG benchmarks and attack settings.

In summary, TrustRAG enhances the performance of RAG systems by implementing a two-stage defense mechanism that 
filters compromised and irrelevant content, improving the resistance of RAG systems to attacks and maintaining high
response accuracy.

Sources:

* TrustRAG: Enhancing Robustness and Trustworthiness in RAG, Huichi Zhou, Kin-Hei Lee, Zhonghao Zhan, Yue Chen, 
Zhenhao Li.
* Retrieval augmented generation. Zhou et al., 2024.
* Large language models. Lewis et al., 2020; Guu et al., 2020; Izacard et al., 2023.
* RAG methods. Zheng et al., 2023; Dai et al., 2022; Glass et al., 2022; Chen et al.

QA Pair 2
Question: How can gender stereotypes in movie recommender systems lead to privacy issues and what kind of broader 
implications can this have on digital computational science?

RAG Answer: Gender stereotypes in movie recommender systems can lead to privacy issues in a couple of ways. First, 
based on users' demographic data, demographic biases such as gender bias may be introduced into the system for 
business purposes [42, 43, 44]. For example, action movies are often promoted more to male audiences, while 
romantic animated movies are marketed with themes that attract female audiences. These practices can create 
gender-specific viewing patterns, reducing the variety of genres each gender gets to see and possibly restricting 
individual preferences. 

Moreover, the potential for gender bias in the system can also result in issues such as filter bubbles or feedback 
loop effects. These phenomena confine users to repetitive, similar recommendations, thereby limiting their exposure
to diverse content and reinforcing stereotypical behavior [45]. 

In a specific attack scenario, an attacker can determine users' gender, a private attribute, by exploiting gender 
stereotypes about movie preferences and analyzing users' feedback data, which is either publicly available or 
observed within the system [42, 44]. This is done in two phases. In the first phase, a user study involving 630 
participants identified gender stereotypes associated with movie genres, which often influence viewing choices. In 
the second phase, four inference algorithms were applied to detect gender stereotypes by combining the findings 
from the first phase with users' feedback data [42, 44]. Results showed that these algorithms performed more 
effectively than relying solely on feedback data for gender inference. 

The broader implications of this on digital computational science are significant. The quantification of gender 
stereotypes' extent can evaluate their broader impact on digital computational science [42, 44]. The presence of 
gender stereotypes in movie recommender systems can perpetuate prevailing social norms, leading to a lack of 
diversity in content exposure and potentially reinforcing harmful stereotypes. Therefore, addressing these issues 
is crucial for maintaining user privacy and promoting diversity and inclusivity in digital computational science.

(Cited Sources: [42, 43, 44, 45] from A Closer Look on Gender Stereotypes in Movie Recommender Systems and Their 
Implications with Privacy)

QA Pair 3
Question: How does TrustRAG, a defense framework for Retrieval-Augmented Generation (RAG) systems, enhance 
robustness and trustworthiness in the face of corpus poisoning attacks?

RAG Answer: TrustRAG is a defense framework designed to enhance the robustness and trustworthiness of 
Retrieval-Augmented Generation (RAG) systems in the face of corpus poisoning attacks. It does this by employing a 
two-stage defense mechanism. 

In the first stage, TrustRAG uses K-means clustering to identify potential attack patterns in retrieved documents 
based on their semantic embeddings. This helps to isolate suspicious content that may be malicious. 

The second stage of TrustRAG involves implementing a mechanism to filter compromised and irrelevant content before 
it reaches the language model. This is done through a majority-vote mechanism across retrieved documents and 
carefully engineered prompts. 

This approach is designed to maintain robust and trustworthy responses even when multiple malicious documents have 
contaminated the retrieval corpus. It's important to note that previous approaches to mitigate noisy information 
through majority-vote mechanisms across retrieved documents have become ineffective when attackers inject multiple 
malicious documents that outnumber benign ones. 

TrustRAG is proposed in the paper "TrustRAG: Enhancing Robustness and Trustworthiness in RAG" by Huichi Zhou, 
Kin-Hei Lee, Zhonghao Zhan, Yue Chen, and Zhenhao Li in 2024.

<br>

### **Step 5:** Implement A Human Preference Metric

In this section, we can implement the fourth part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- **Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."**

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [9]:
## TODO: Adapt this prompt for whichever LLM you're actually interested in using. 
## If it's llama, maybe system message would be good?
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION: 
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
The Score must be either 1 or 2.
Score must be 1 if the second answer lies, does not answer the question, or is inferior to the first answer.
Score must be 2 if the The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION: 
""")

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How does TrustRAG, a robust framework, enhance the performance of Retrieval-Augmented 
Generation (RAG) systems and improve their resistance to attacks?

Synth Answer: Answer: TrustRAG is a two-stage defense mechanism designed to strengthen the robustness and 
trustworthiness of RAG systems. It employs K-means clustering to identify potential attack patterns in retrieved 
documents based on their semantic embeddings, effectively isolating suspicious content. Additionally, it uses 
cosine similarity and ROUGE metrics to detect malicious documents and resolve discrepancies between the model's 
internal knowledge and external information through a self-assessment process. TrustRAG functions as a 
plug-and-play, training-free module that integrates with any language model, maintaining high contextual relevance 
while providing substantial improvements in retrieval accuracy, efficiency, and attack resistance compared to 
existing approaches. It has been shown to improve runtime performance, achieving a runtime approximately two times 
greater than Vanilla RAG, which is a reasonable trade-off for the significant improvements in robustness and 
reliability offered by TrustRAG.

RAG Answer: TrustRAG is a robust framework designed to enhance the performance of Retrieval-Augmented Generation 
(RAG) systems and improve their resistance to attacks. According to the abstract of "TrustRAG: Enhancing Robustness
and Trustworthiness in RAG," the framework aims to address the vulnerabilities of RAG systems to corpus poisoning 
attacks, which can significantly degrade the performance of large language models (LLMs) through the injection of 
malicious content.

TrustRAG implements a two-stage defense mechanism to filter compromised and irrelevant content before it reaches 
the language model. The first stage employs K-means clustering to identify potential attack patterns in retrieved 
documents based on their semantic embeddings, effectively isolating suspicious content.

The second stage involves the use of an objective function to evaluate the quality of the answer produced by the 
LLM based on the k retrieved texts for the target question. The objective function is large when the answer 
produced by the LLM is the target answer, indicating that the retrieved texts are relevant and trustworthy.

According to the paper, TrustRAG conducts an extensive evaluation on multiple knowledge databases and LLMs, 
comparing it with advanced RAG systems. The results show that TrustRAG decreases the attack success rate while 
maintaining high response accuracy on different popular RAG benchmarks and attack settings.

In summary, TrustRAG enhances the performance of RAG systems by implementing a two-stage defense mechanism that 
filters compromised and irrelevant content, improving the resistance of RAG systems to attacks and maintaining high
response accuracy.

Sources:

* TrustRAG: Enhancing Robustness and Trustworthiness in RAG, Huichi Zhou, Kin-Hei Lee, Zhonghao Zhan, Yue Chen, 
Zhenhao Li.
* Retrieval augmented generation. Zhou et al., 2024.
* Large language models. Lewis et al., 2020; Guu et al., 2020; Izacard et al., 2023.
* RAG methods. Zheng et al., 2023; Dai et al., 2022; Glass et al., 2022; Chen et al.

Synth Evaluation: [2] Both answers provide a detailed explanation of TrustRAG and its function as a robust 
framework for enhancing the performance of RAG systems and improving their resistance to attacks. Answer 2 provides
additional information about TrustRAG's two-stage defense mechanism and its evaluation against advanced RAG 
systems, making it slightly better than Answer 1. However, Answer 2 does not introduce any inconsistencies with the
ground truth answer, so it does not warrant a score of 1.

Set 2

Question: Question: How can gender stereotypes in movie recommender systems lead to privacy issues and what kind of
broader implications can this have on digital computational science?

...

Synth Evaluation: [2] Both answers provide a comprehensive explanation of how gender stereotypes in movie 
recommender systems can lead to privacy issues and have broader implications for digital computational science. The
second answer even provides more specific details about the attack scenario and cites sources. It does not 
introduce any inconsistencies and is better structured, making it a more preferable answer.

Set 3

Question: Question: How does TrustRAG, a defense framework for Retrieval-Augmented Generation (RAG) systems, 
enhance robustness and trustworthiness in the face of corpus poisoning attacks?

Synth Answer: Answer: TrustRAG is a two-stage defense mechanism aimed at countering attacks involving multiple 
maliciously injected documents in RAG systems. It employs K-means filtering to reduce the presence of malicious 
documents and uses both internal and external knowledge sources to resolve conflicts and mitigate the impact of 
these attacks. By comprehensively evaluating TrustRAG across benchmarks, it has been demonstrated that it 
outperforms existing defenses, maintaining high accuracy even under aggressive poisoning scenarios where 
traditional approaches fail. TrustRAG's plug-and-play architecture allows for immediate deployment with any Large 
Language Models (LLMs).

RAG Answer: TrustRAG is a defense framework designed to enhance the robustness and trustworthiness of 
Retrieval-Augmented Generation (RAG) systems in the face of corpus poisoning attacks. It does this by employing a 
two-stage defense mechanism. 

In the first stage, TrustRAG uses K-means clustering to identify potential attack patterns in retrieved documents 
based on their semantic embeddings. This helps to isolate suspicious content that may be malicious. 

The second stage of TrustRAG involves implementing a mechanism to filter compromised and irrelevant content before 
it reaches the language model. This is done through a majority-vote mechanism across retrieved documents and 
carefully engineered prompts. 

This approach is designed to maintain robust and trustworthy responses even when multiple malicious documents have 
contaminated the retrieval corpus. It's important to note that previous approaches to mitigate noisy information 
through majority-vote mechanisms across retrieved documents have become ineffective when attackers inject multiple 
malicious documents that outnumber benign ones. 

TrustRAG is proposed in the paper "TrustRAG: Enhancing Robustness and Trustworthiness in RAG" by Huichi Zhou, 
Kin-Hei Lee, Zhonghao Zhan, Yue Chen, and Zhenhao Li in 2024.

Synth Evaluation: [2] The second answer is more informative as it explains the two stages of TrustRAG in detail. It
also highlights how previous methods have become ineffective and distinguishes how TrustRAG addresses this issue. 
Moreover, it provides a citation for the paper where TrustRAG was proposed. The answer does not introduce any 
inconsistencies with the ground truth.

<br>

**Congratulations! We now have an LLM system that reasons about our pipeline and tries to evaluate it!** Now that we have some judge results, we can simply aggregate the results and see how often our formulation was according to an LLM:

In [10]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 1.0


In [17]:

from langserve import RemoteRunnable
from langchain_core.output_parsers import StrOutputParser

llm = RemoteRunnable("http://0.0.0.0:9012/basic_chat/") | StrOutputParser()
for token in llm.stream("Hello World! How is it going?"):
    print(token, end='')


Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today? Is there something you would like to know or a problem you need help with?

In [34]:
retrieval_chain9012 = RemoteRunnable("http://0.0.0.0:9012/retriever/") | StrOutputParser()
generator_chain9012 = RemoteRunnable("http://0.0.0.0:9012/generator/")
rag_chain9012 = retrieval_chain9012 | generator_chain9012



In [38]:
# for token in retrieval_chain9012.stream("Tell me something interesting about RAG"):
#     print(token, end='')


----

<br>

## **Part 4:** Advanced Formulations

The exercise above was meant to prepare you for the final assessment of the course and showcased a simple but effective evaluator chain. The objective and implementation details were provided for you, and the logic for using it probably makes sense now that you've seen it in action. 

With that being said, this metric was merely a product of us specifying:
- **What kind of behavior is important for our pipeline to have?**
- **What do we need to do in order to exhibit and evaluate this behavior?**

From these two questions, we could have come up with plenty of other evaluation metrics that could have assessed different attributes, incorporated different evaluator chain techniques, and even required different pipeline organization strategies. Though far from an exhaustive list, some common formulations you will likely come across may include:

- **Style Evaluation:** Some evaluation formulations can be as simple as "let me ask some questions and see if the output feels desirable." This might be used to see whether a chatbot "acts like it's supposed to" based on a description provided to a judge LLM. We're using quotations since this kind of assessment can reasonably be achieved with nothing but prompt engineering and a while loop.

- **Ground-Truth Evaluation:** In our chain, we used synthetic generation to create some random questions and answers using a sampling strategy, but in reality you may actually have some representative questions and answers that you need your chatbot to consistently get right! In this case, a modification of the exercise chain above should be implemented and closely monitored as you develop your pipelines.

- **Retrieval/Augmentation Evaluation:** This course made many assumptions about what kinds of preprocessing and prompting steps would be good for your pipelines, and much of this was determined by experimentation. Factors such as document preprocessing, chunking strategies, model selection, and prompt specification all played important roles, so creating metrics to validate these decisions may be of interest. This kind of metric might require your pipeline to output your context chunks or may even rely solely on embedding similarity comparisons, so keep this in mind when trying to implement a chain that works with multiple evaluation strategies. Consider the [**RagasEvaluatorChain**](https://docs.ragas.io/en/stable/howtos/integrations/langchain.html) abstraction as a decent starting point for making an custom generalizable evaluation routine. 

- **Trajectory Evaluation:** Using more advanced agent formulations, you can implement multiple-query strategies that assume the presence of conversational memory. With this, you can implement an evaluation agent which can:
    - Ask a series of questions in order to evaluate how well the agent is able to adapt and cater to the scenario. This kind of system generally considers a series of correspondence and aims to tease out and evaluate a "trajectory" of how the agent navigated the conversation. The [**LangChain Trajectory Evaluators documentation**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/trajectory/) is a good starting point.
    - Alternatively, you could also implement an evaluation agent that tries to achieve objectives by interacting with the chatbot. Such an agent can output whether they were able to navigate to their solution in a natural manner, and can even be used to generate a report about the percieved performance. The [**LangChain Agents documentation**](https://python.langchain.com/v0.1/docs/modules/agents/) is a good starting point!

<br>

At the end of the day, just make sure to use the tools you have at your disposal appropriately. By this point in the course, you should already be well-acquainted with the LLM core value propositions: **They're powerful, scalable, predictable, controllable, and orchestratable... but will act unpredictably when you just expect them to work by default.** Assess your needs, formulate and validate your pipelines, give enough information, and add as much control as you can to make your system work consistently, efficiently, and effectively.

----

<br>

## **Part 5: [Assessment]** Evaluating For Credit

Welcome to the last exercise of the course! Hopefully you've enjoyed the material and are ready to actually get credit for these notebooks! For this part:

- **Make sure you're in the course environment**
- **Make sure `docstore_index/` has been uploaded to the course environment...**
    - **...and contains [at least one Arxiv paper](https://arxiv.org/search/advanced) which has been updated recently.**
- **Make sure you don't have some old session of [`09_langserve.ipynb`](09_langserve.ipynb) already occupying the port. Your assessment requires you to implement the new `/retriever` and `/generator` endpoints!!**

**Objective:** On launch, [**`frontend/frontend_block.py`**](frontend/frontend_block.py) had several lines of code which trigger the course pass condition. Your objective is to invoke that series of commands by using your pipeline to pass the **Evaluation** check! Recall [`09_langserve.ipynb`](09_langserve.ipynb) and use it as a starting example! As a recommendation, consider duplicating it so that you can keep the original as an authoritative reference. 

**Once Finished:** While your course environment is still open, please navigate back to your course environment launcher area and click the **"Assess Task"** button! After that, you're all done!

In [ ]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';

----

<br>

## <font color="#76b900">**Congratulations On Completing The Course**</font>

Hopefully this course was not only exciting and challenging, but also adequately prepared you for work on the cutting edge of LLM and RAG system development! Going forward, you should have the skills necessary to tackle industry-level challenges and explore RAG deployment with open-source models and frameworks.

**Some NVIDIA-specific releases related to this that you may find interesting include:**
- [**NVIDIA NIM**](https://www.nvidia.com/en-us/ai/), which offers microservice spinup routines that can be deployed on local compute.
- [**TensorRT-LLM**](https://github.com/NVIDIA/TensorRT-LLM) is the current recommended framework for deploying GPU-accelerated LLM model engines in production settings.
- [**NVIDIA's Generative AI Examples Repo**](https://github.com/NVIDIA/GenerativeAIExamples), which includes the current canonical microservice example application and will be updated with new resources as new production workflows get released.
- [**The Knowledge-Based Chatbot Technical Brief**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief) which discusses additional publicly-accessible details on productionalizing RAG systems.

**Additionally, some key topics you may be interested in delving more into include:**
- [**LlamaIndex**](https://www.llamaindex.ai/), which has strong components that can augment and occasionally improve upon the LangChain RAG features.
- [**LangSmith**](https://docs.smith.langchain.com/), an upcoming agent productionalization service offered by LangChain.
- [**Gradio**](https://www.gradio.app/), though touched on in the course, has many more interface options which will be worth investigating. For inspiration, consider checking out [**HuggingFace Spaces**](https://huggingface.co/spaces) for examples.
- [**LangGraph**](https://python.langchain.com/docs/langgraph/) is a framework for graph-based LLM orchestration, and is a natural next step forward for those interested in [multi-agent workflows](https://blog.langchain.dev/langgraph-multi-agent-workflows/).
- [**DSPy**](https://github.com/stanfordnlp/dspy), a flow engineering framework that allows you to optimize LLM orchestration pipelines based on empirical performance results.

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>